In [2]:
import pandas as pd

In [5]:
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [7]:
df_financial = pd.read_csv('financial_data.csv')
df_prolong = pd.read_csv('prolongations.csv')

Тут мы считываем данные в формат Data Frame из файлов financial_data.csv и prolongations.csv в df_financial и df_prolong  

In [10]:
df_financial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             451 non-null    int64 
 1   Причина дубля  301 non-null    object
 2   Ноябрь 2022    156 non-null    object
 3   Декабрь 2022   159 non-null    object
 4   Январь 2023    139 non-null    object
 5   Февраль 2023   145 non-null    object
 6   Март 2023      168 non-null    object
 7   Апрель 2023    174 non-null    object
 8   Май 2023       190 non-null    object
 9   Июнь 2023      190 non-null    object
 10  Июль 2023      195 non-null    object
 11  Август 2023    199 non-null    object
 12  Сентябрь 2023  186 non-null    object
 13  Октябрь 2023   182 non-null    object
 14  Ноябрь 2023    171 non-null    object
 15  Декабрь 2023   146 non-null    object
 16  Январь 2024    95 non-null     object
 17  Февраль 2024   101 non-null    object
 18  Account        451 non-null   

In [11]:
df_prolong.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477 entries, 0 to 476
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      477 non-null    int64 
 1   month   477 non-null    object
 2   AM      477 non-null    object
dtypes: int64(1), object(2)
memory usage: 11.3+ KB


In [12]:
df_financial.head(6)

,id,Причина дубля,Ноябрь 2022,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023,Май 2023,Июнь 2023,Июль 2023,Август 2023,Сентябрь 2023,Октябрь 2023,Ноябрь 2023,Декабрь 2023,Январь 2024,Февраль 2024,Account
0,42,NaN,"36 220,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
1,657,первая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
2,657,вторая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
3,594,NaN,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
4,665,NaN,"10 000,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
5,637,NaN,"38 045,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Соколова Анастасия Викторовна


In [13]:
def parse_number(x):
    if isinstance(x, str):
        x = x.strip().lower()
        if x in ['стоп', 'end']:
            return 'стоп'
        if x in ['в ноль']:
            return 'в ноль'
        x = x.replace(',', '.')
        try:
            return float(x)
        except:
            return 0
    elif pd.isna(x):
        return 0
    return x

Использую функцию isinstance, чтоб узнать, принадлежит ли объект определенному типу данных

Далее привожу x к единому формату  и на всякий случай делаю все символы строчными 

Если x встречается как стоп или end, то возвращаем стоп. Аналогично для ноль. Ну а все остальные значения это числа. В них заменяю запятые на точки и удаляю все пробелы внутри. В случае работы без ошибок возвращаем вещественный тип данных, иначе возвращаем ноль. Также возвращаем ноль, если значение NaN. Но если это не строка и не NaN, то просто возвращаем x

Эта функция приводит всех значений таблицы к единому формату

In [14]:
month_cols = [col for col in df_financial.columns if re.match(r'[А-Яа-я]+\s\d{4}', col)]
for col in month_cols:
   df_financial[col] = df_financial[col].apply(parse_number)

Переформатирую числа
В month_cols получаю список столбцов с результатами отгрузки

А apply позволяет в DataFrame применить функцию к колонке

In [15]:
merged = df_prolong.merge(df_financial, on='id', how='left')

ДЛя удобства делаю оба DataFrame в одну таблицу

In [16]:
months = [
    'Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь',
    'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь'
]
month_name_to_number = {name.lower(): i + 1 for i, name in enumerate(months)}
month_number_to_name = {i + 1: name for i, name in enumerate(months)}

month_name_to_number это список в котором для каждого номера хранится его номер

month_number_to_name противоположный список, то есть для каждого номера хранится свой месяц 

In [17]:
def get_adjusted_value(row, target_month, later_months, previous_month):
    value = row.get(target_month)
    if value == 'стоп':
        return None  # не учитываем проекты со статусом "стоп"
    if value == 'в ноль':
        only_zeros = all(
            row.get(m, 0) in [0, 'в ноль'] for m in later_months
        )
        if only_zeros:
            return row.get(previous_month, 0)
        else:
            return 0
    return value if value != 'в ноль' else 0

get_adjusted_value это функция, которая обрабатывает значения "стоп" и "в ноль"

In [18]:
merged.head()

,id,month,AM,Причина дубля,Ноябрь 2022,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023,...,Июнь 2023,Июль 2023,Август 2023,Сентябрь 2023,Октябрь 2023,Ноябрь 2023,Декабрь 2023,Январь 2024,Февраль 2024,Account
0,42,ноябрь 2022,Васильев Артем Александрович,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Васильев Артем Александрович
1,453,ноябрь 2022,Васильев Артем Александрович,NaN,в ноль,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Васильев Артем Александрович
2,548,ноябрь 2022,Михайлов Андрей Сергеевич,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Михайлов Андрей Сергеевич
3,87,ноябрь 2022,Соколова Анастасия Викторовна,основные работы,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Иванова Мария Сергеевна
4,429,ноябрь 2022,Соколова Анастасия Викторовна,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Соколова Анастасия Викторовна


In [33]:


def calculate_coefficients(df, current_month):
    coefficients = []

    try:
        month_str, year_str = current_month.strip().split()
        current_dt = datetime(int(year_str), month_name_to_number[month_str.lower()], 1)
    except Exception as e:
        print(f"Ошибка парсинга месяца {current_month}: {e}")
        return pd.DataFrame()

    # Определение нужных месяцев
    prev_dt = current_dt - relativedelta(months=1)  # апрель, если текущий — май
    prev2_dt = current_dt - relativedelta(months=2)  # март, если текущий — май

    prev_month = f"{month_number_to_name[prev_dt.month]} {prev_dt.year}"
    prev2_month = f"{month_number_to_name[prev2_dt.month]} {prev2_dt.year}"
    current_month_str = f"{month_number_to_name[current_dt.month]} {current_dt.year}"

    later_months_from_prev = [
        f"{month_number_to_name[(prev_dt.month + i - 1) % 12 + 1]} {prev_dt.year + ((prev_dt.month + i - 1) // 12)}"
        for i in range(1, 13)
    ]
    later_months_from_prev2 = [
        f"{month_number_to_name[(prev2_dt.month + i - 1) % 12 + 1]} {prev2_dt.year + ((prev2_dt.month + i - 1) // 12)}"
        for i in range(1, 13)
    ]

    for account, group in df.groupby('Account'):
        k1_num = 0
        k1_den = 0
        k2_num = 0
        k2_den = 0

        for _, row in group.iterrows():
            # Пролонгация в 1-й месяц после окончания
            end_val = get_adjusted_value(row, prev_month, later_months_from_prev, '')
            next_val = get_adjusted_value(row, current_month_str, [], prev_month)
            if end_val is not None and isinstance(end_val, (int, float)) and end_val > 0:
                k1_num += next_val if isinstance(next_val, (int, float)) else 0
                k1_den += end_val

            # Пролонгация во 2-й месяц после окончания
            end2_val = get_adjusted_value(row, prev2_month, later_months_from_prev2, '')
            skip_val = get_adjusted_value(row, prev_month, later_months_from_prev, prev2_month)
            second_val = get_adjusted_value(row, current_month_str, [], prev_month)

            if end2_val is not None and isinstance(end2_val, (int, float)) and end2_val > 0 and (not skip_val or skip_val == 0):
                k2_num += second_val if isinstance(second_val, (int, float)) else 0
                k2_den += end2_val

        k1 = round(k1_num / k1_den, 2) if k1_den else None
        k2 = round(k2_num / k2_den, 2) if k2_den else None

        coefficients.append({
            'Account': account,
            'K1': k1,
            'K2': k2
        })

    return pd.DataFrame(coefficients)

На вход функции приходит наша таблица на одного менеджера - df, и current_month - месяц, для которого анализируем пролонгацию.

In [40]:
# Список месяцев
months_to_analyze = sorted(set(df_prolong['month']))

# Расчёт коэффициентов
results = []
for month in months_to_analyze:
    for manager in merged['AM'].dropna().unique():
        manager_data = merged[merged['AM'] == manager]
        coeffs = calculate_coefficients(manager_data, month)

        if not coeffs.empty:
            avg_k1 = coeffs['K1'].mean(skipna=True)
            avg_k2 = coeffs['K2'].mean(skipna=True)

            results.append({
                'Месяц': month,
                'Менеджер': manager,
                'K1': avg_k1,
                'K2': avg_k2
            })


In [43]:
final_df = pd.DataFrame(results)

# Группировки
final_df['Год'] = final_df['Месяц'].str.extract(r'(\d{4})')
yearly_report = final_df.groupby(['Год', 'Менеджер'])[['K1', 'K2']].mean().reset_index()
overall_report = final_df.groupby('Менеджер')[['K1', 'K2']].mean().reset_index()

overall_summary = pd.DataFrame({
    'Менеджер': ['Весь отдел'],
    'K1': [final_df['K1'].mean(skipna=True)],
    'K2': [final_df['K2'].mean(skipna=True)]
})


# Сохранение
final_df.to_csv('prolongation_report_fixed.csv', index=False)
yearly_report.to_csv('yearly_report.csv', index=False)
overall_report.to_csv('overall_report.csv', index=False)
overall_summary.to_csv('department_summary.csv', index=False)

print("Пример данных:")
print(final_df.head())

Пример данных:
         Месяц                       Менеджер    K1   K2   Год
0  август 2023   Васильев Артем Александрович   NaN  NaN  2023
1  август 2023      Михайлов Андрей Сергеевич   NaN  NaN  2023
2  август 2023  Соколова Анастасия Викторовна  0.67  NaN  2023
3  август 2023        Иванова Мария Сергеевна   NaN  NaN  2023
4  август 2023    Попова Екатерина Николаевна   NaN  0.0  2023
